## **Text Feature Engineering and Modelling**
Done by Wong Wen Bing 230436M

In [5]:
pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 4.0 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [17]:
import nltk
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [11]:
df=pd.read_csv('230436M_cleaned_data.csv')
df.head()

,Unnamed: 0,index,description,species,cleaned_text
0,0,0,2 Jun 2023 ï¿½ The Javan myna shares some simi...,Javan Myna,"['2', 'jun', '2023', 'javan', 'myna', 'shares'..."
1,1,1,Click here for more information about the Red ...,Collared Kingfisher,"['click', 'information', 'red', 'list', 'categ..."
2,2,2,The black-headed oriole ( Oriolus larvatus) is...,Black-naped Oriole,"['blackheaded', 'oriole', 'oriolus', 'larvatus..."
3,3,3,"Search from thousands of royalty-free ""Javan M...",Javan Myna,"['search', 'thousands', 'royaltyfree', 'javan'..."
4,4,4,521 foreground recordings and 156 background ...,Little Egret,"['521', 'foreground', 'recordings', '156', 'ba..."


In [7]:
y=df['species']
x=df['cleaned_text']

In [14]:
display(x.shape)
display(y.shape)

X_train, X_test, y_train, y_test=train_test_split(x, y, test_size=0.2, random_state=42)

(611,)

(611,)

In [19]:
rf=RandomForestClassifier()
rf.fit(X_train,y_train)

ValueError: could not convert string to float: "['research', '100', 'million', 'voter', 'records', 'free']"